In [1]:
import os
import sys
sys.path.append('..')

In [2]:
import import_ipynb
from utils.dataset_loader import CreateDataset

importing Jupyter notebook from ..\utils\dataset_loader.ipynb


In [3]:
import torch
from torch import nn
from torch import optim

import random
import numpy as np

from tqdm import tqdm

In [17]:
### cpu, gpu 선택
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### 불용어 사용 여부
use_stopword = True

### batch_size
batch_size = 32

In [5]:
### 미리 만들어둔 데이터셋을 가져옴
dataset = CreateDataset(device=device, use_stopword=use_stopword)

### 데이터셋에서 iterator만 뽑아냄
train_iterator, valid_iterator, test_iterator = dataset.get_iterator(batch_size=batch_size)

In [6]:
### Encoder 단순하게 LSTM으로만 이루어져 있음
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

In [7]:
### Decoder 단순하게 LSTM으로만 이루어져 있으며, Encoder로 부터 context vector를 전달 받음(hidden, cell)
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
    def forward(self, x, hidden, cell):
        x = x.unsqueeze(0)
        embedded = self.dropout(self.embedding(x))
        outputs, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        output = self.fc_out(outputs).squeeze(0)
        return output, hidden, cell

In [8]:
### Encoder는 한번에 학습이 가능하지만 Decoder는 recursive하게 하나씩 예측해야한다.
### 학습시에 모든 label 데이터를 넣어서 output를 뽑아내어 for문 없이 한번에 처리 할 수 있지만
### 하나씩 예측하며 예측값을 가지고 다음 step의 token을 예측하는 방식으로 이용하고 있다,
### Inference 시에도 해당 코드 이용가능
class Seq2Seq(nn.Module):
    def __init__(self, enc, dec, device):
        super().__init__()
        self.enc = enc
        self.dec = dec
        self.device = device
        self.output_dim = dec.output_dim
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        hidden, cell = self.enc(src)
        
        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        output_dim = self.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, output_dim).to(self.device)
        
        dec_input = trg[0]
        for t in range(1, trg_len):
            output, hidden, cell = self.dec(dec_input, hidden, cell)
            outputs[t] = output
            
            top1 = torch.argmax(output, dim=1)
            
            dec_input = top1 if random.random() > teacher_forcing_ratio else trg[t]
            
        return outputs

In [9]:
def train(model, criterion, optimizer, iterator, clip):
    model.train()
    cost = []
    for batch in tqdm(iterator):
        optimizer.zero_grad()
        src = batch.src
        trg = batch.trg
        outputs = model(src, trg)
            
        output_dim = outputs.shape[-1]

        outputs = outputs[1:].reshape(-1, output_dim)
        target = trg[1:].reshape(-1).long()
        loss = criterion(outputs, target)
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) ## 기울기 값이 clip 값을 초과하지 않도록 제한을 둠
        
        cost += [loss.item()]
        loss.backward()
        
        optimizer.step()
    return np.mean(cost)

In [10]:
def evaluation(model, criterion, iterator):
    model.eval()
    cost = []
    with torch.no_grad():
        for batch in tqdm(iterator):
            src = batch.src
            trg = batch.trg
            outputs = model(src, trg)
            
            output_dim = outputs.shape[-1]
            
            outputs = outputs[1:].reshape(-1, output_dim)
            target = trg[1:].reshape(-1).long()
            
            loss = criterion(outputs, target)
            
            cost += [loss.item()]
            
    return np.mean(cost)

In [11]:
input_dim = len(dataset.SRC.vocab)
output_dim = len(dataset.TRG.vocab)
emb_dim = 256
hid_dim = 512
n_layers = 2
dropout = 0.1

In [12]:
enc = Encoder(input_dim, emb_dim, hid_dim, n_layers, dropout).to(device)
dec = Decoder(output_dim, emb_dim, hid_dim, n_layers, dropout).to(device)
model = Seq2Seq(enc, dec, device).to(device)
epochs = 10

In [13]:
pad_index = dataset.SRC.vocab.stoi[dataset.SRC.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index=pad_index)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [14]:
model

Seq2Seq(
  (enc): Encoder(
    (embedding): Embedding(18668, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.1)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec): Decoder(
    (embedding): Embedding(9799, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.1)
    (dropout): Dropout(p=0.1, inplace=False)
    (fc_out): Linear(in_features=512, out_features=9799, bias=True)
  )
)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 19,670,855 trainable parameters


In [16]:
for epoch in range(epochs):
    train_loss = train(model, criterion, optimizer, train_iterator, 1)
    eval_loss = evaluation(model, criterion, valid_iterator)
    print(train_loss, eval_loss)

  0%|                                                                                  | 1/907 [00:00<01:43,  8.77it/s]

5.216307528620912 4.858572691679001


  0%|                                                                                  | 1/907 [00:00<02:00,  7.52it/s]

4.689778715404944 4.520121172070503


  0%|                                                                                  | 1/907 [00:00<01:36,  9.35it/s]

...

3.630755118524621 3.653802067041397


  0%|▏                                                                                 | 2/907 [00:00<01:17, 11.70it/s]

3.510580916651817 3.5038353875279427


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 39.46it/s]

3.4232479311534036 3.5013695508241653
